In [ ]:
def DIEN(feature_columns, behavior_feature_list, behavior_seq_feature_list, neg_seq_feature_list, use_neg_sample=False, alpha=1.0):
    # 构建输入层: 输出{feature_name: keras_layer}
    input_layer_dict = build_input_layers(feature_columns)
    
    # 将Input层转化为列表的形式作为model的输入
    input_layers = list(input_layer_dict.values())       # 各个输入层
    user_behavior_length = input_layer_dict["hist_len"]
    
    # 筛选出特征中的sparse_fea, dense_fea, varlen_fea
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeat), feature_columns)) if feature_columns else []
    dense_feature_columns = list(filter(lambda x: isinstance(x, DenseFeat), feature_columns)) if feature_columns else []
    varlen_sparse_feature_columns = list(filter(lambda x: isinstance(x, VarLenSparseFeat), feature_columns)) if feature_columns else []
    
    # 获取dense
    dnn_dense_input = []# [keras_layer1, keras_layer2,keras_layer3]
    for fc in dense_feature_columns:
        dnn_dense_input.append(input_layer_dict[fc.name])

    # 将所有的dense特征拼接
    dnn_dense_input = concat_input_list(dnn_dense_input)

    # 构建embedding字典
    embedding_layer_dict = build_embedding_layers(feature_columns, input_layer_dict)
    
    # 因为这里最终需要将embedding拼接后直接输入到全连接层(Dense)中, 所以需要Flatten
    dnn_sparse_embed_input = concat_embedding_list(sparse_feature_columns, input_layer_dict, embedding_layer_dict, flatten=True)
    # 将所有sparse特征的embedding进行拼接
    dnn_sparse_input = concat_input_list(dnn_sparse_embed_input)
    
    # 获取当前的行为特征(movie)的embedding，这里有可能有多个行为产生了行为序列，所以需要使用列表将其放在一起
    query_embed_list = embedding_lookup(behavior_feature_list, input_layer_dict, embedding_layer_dict)
    # 获取行为序列(movie_id序列, hist_movie_id) 对应的embedding，这里有可能有多个行为产生了行为序列，所以需要使用列表将其放在一起
    keys_embed_list = embedding_lookup(behavior_seq_feature_list, input_layer_dict, embedding_layer_dict)
    # 把q,k的embedding拼在一块
    query_emb, keys_emb = concat_input_list(query_embed_list), concat_input_list(keys_embed_list)
    
    # 采样的负行为
    neg_uiseq_embed_list = embedding_lookup(neg_seq_feature_list, input_layer_dict, embedding_layer_dict)
    neg_concat_behavior = concat_input_list(neg_uiseq_embed_list)
    
    # 兴趣进化层的计算过程
    dnn_seq_input, aux_loss = interest_evolution(keys_emb, query_emb, user_behavior_length, neg_concat_behavior, gru_type="AUGRU")
    
    # 后面的全连接层
    deep_input_embed = Concatenate()([dnn_dense_input, dnn_sparse_input, dnn_seq_input])
    
    # 获取最终dnn的logits
    dnn_logits = get_dnn_logits(deep_input_embed, activation='prelu')
    model = Model(input_layers, dnn_logits)
    
    # 加兴趣提取层的损失  这个比例可调
    if use_neg_sample:
        model.add_loss(alpha * aux_loss)
        
    # 所有变量需要初始化
    tf.compat.v1.keras.backend.get_session().run(tf.compat.v1.global_variables_initializer())
    return model
